In [3]:
import requests
import pandas as pd
import numpy as np
import folium
from folium.plugins import MiniMap

In [4]:
## region에는 검색명이 들어갈 것임
## page_num은 1~3이 입력될 건데, 한 페이지 당 검색목록이 최대 15개임.
## 만약 page_num이 4이상이 되면 3페이지랑 같은 15개의 결과 값을 가져옴. 그래서 1~3만 쓰는 것임
## 입력 예시 -->> headers = {"Authorization": "KakaoAK f221u3894o198123r9"}
## ['meta']['total_count']은 내가 검색했을 때, 나오는 총 결과 값. 
## ['meta']['total_count']이 45보다 크면 45개만 가져오게 됨

def elec_location(region,page_num):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
    params = {'query': region,'page': page_num}
    headers = {"Authorization": "KakaoAK 98a509997b30ad1fd8ae661106f85dc9"}

    places = requests.get(url, params=params, headers=headers).json()['documents']
    total = requests.get(url, params=params, headers=headers).json()['meta']['total_count']
    if total > 45:
        print(total,'개 중 45개 데이터밖에 가져오지 못했습니다!')
    else :
        print('모든 데이터를 가져왔습니다!')
    return places


In [5]:
## 이 함수는 위 함수 결과 값(1 ~ 45개) 하나하나 분리해서 저장할 것임
## 1번 결과 값 안에는 1번 충전소 이름, 위도, 경도, 전화번호, 도로명 주소 등이 있는데 각각 배열에 저장
## 우리는 충전소 ID, 충전소 이름, 위도, 경도, 도로명주소, 사이트주소를 저장할 것임

def elec_info(places):
    X = []
    Y = []
    stores = []
    road_address = []
    place_url = []
    ID = []
    i = 0
    for place in places:
        i += 1
        print(i)
        X.append(float(place['x']))
        Y.append(float(place['y']))
        stores.append(place['place_name'])
        road_address.append(place['road_address_name'])
        place_url.append(place['place_url'])
        ID.append(place['id'])
        if i > 14:
            break

    ar = np.array([ID,stores, X, Y, road_address,place_url]).T
    df = pd.DataFrame(ar, columns = ['ID','stores', 'X', 'Y','road_address','place_url'])
    return df

In [6]:
## 여러개의 키워드를 검색할 때 사용할 함수임
## location_name에는 ['성산일출봉 전기충전소, '한림공원 전기충전소', ... ,'모슬포 전기충전소']처럼 배열이 입력

def keywords(location_name):
    df = None
    for loca in location:
        for page in range(1,4):
            local_name = elec_location(loca, page)
            local_elec_info = elec_info(local_name)

            if df is None:
                df = local_elec_info
            elif local_elec_info is None:
                continue
            else:
                df = pd.concat([df, local_elec_info],join='outer', ignore_index = True)
    return df

In [7]:
##지도로 보여주기

def make_map(dfs):
    global lat
    global long
    # 지도 생성하기
    m = folium.Map([lat,long],zoom_start=14)

    # 미니맵 추가하기
    minimap = MiniMap() 
    m.add_child(minimap)

    # 마커 추가하기
    for i in range(len(dfs)):
        folium.Marker([df['Y'][i],df['X'][i]],
                  tooltip=dfs['stores'][i],
                  popup=dfs['place_url'][i],
                  ).add_to(m)
    return m

In [10]:

## drop_duplicates를 해주고 인덱스 리셋을 해준다 (중복 검색 가능성 고려)

location = ['남주고등학교']
df = keywords(location)
df = df.drop_duplicates(['ID'])
df = df.reset_index()

lat = [] # 위도
long = [] # 경도
for k in range(len(df['X'])): # 위도데이터와 경도데이터를 하나씩 다시 꺼내서 배열에 쌓음(float으로 변환하기 위함)
    lat.append(df['Y'][k])
    long.append(df['X'][k])
lat = list(map(float, lat))
long = list(map(float, long))

lat = pd.DataFrame(lat).mean()
long = pd.DataFrame(long).mean()

make_map(df)

모든 데이터를 가져왔습니다!
1
2
3
4
모든 데이터를 가져왔습니다!
1
2
3
4
모든 데이터를 가져왔습니다!
1
2
3
4
